## 0. Importar Datos

In [88]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import numpy as np

In [5]:
DataVinos = pd.read_csv('./VinoPinotNoir.csv')
Target = 'Calidad'
DataVinos.head()

,Claridad,Aroma,Cuerpo,Sabor,Fuerza,Calidad,Region
0,1.0,3.3,2.8,3.1,4.1,9.8,1
1,1.0,4.4,4.9,3.5,3.9,12.6,1
2,1.0,3.9,5.3,4.8,4.7,11.9,1
3,1.0,3.9,2.6,3.1,3.6,11.1,1
4,1.0,5.6,5.1,5.5,5.1,13.3,1


## 1. Modelo Completo

Usando las variables mencionadas junto con intersección, tiene el siguiente modelo:
$$ Calidad = \beta_0 + \beta_1*Claridad + \beta_2*Aroma + \beta_3*Cuerpo + \beta_4*Sabor + \beta_5*Fuerza $$

Donde se sustituyen las $\beta_j$ con los valores ajustados:
$$ Calidad = 3.9969 + 2.3395*Claridad + 0.4826*Aroma + 0.2732*Cuerpo + 1.1683*Sabor -0.6840*Fuerza $$

In [7]:
ModeloCompleto = smf.ols(
    f"{Target} ~ Claridad + Aroma + Cuerpo + Sabor + Fuerza",
    DataVinos
).fit()

ModeloCompleto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Calidad   R-squared:                       0.721
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     16.51
Date:                Thu, 18 Sep 2025   Prob (F-statistic):           4.70e-08
Time:                        08:34:18   Log-Likelihood:                -56.378
No. Observations:                  38   AIC:                             124.8
Df Residuals:                      32   BIC:                             134.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.9969      2.232      1.791      0.083      -0.549       8.543
Claridad       2.3395      1.735      1.349      0.187      -1.194       5.873
Aroma          0.4826      0.272      1.771      0.086      -0.072       1.038
Cuerpo         0.2732      0.333      0.821      0.418      -0.404       0.951
Sabor          1.1683      0.304      3.837      0.001       0.548       1.789
Fuerza        -0.6840      0.271     -2.522      0.017      -1.236      -0.132
==============================================================================
Omnibus:                        1.181   Durbin-Watson:                   0.837
Prob(Omnibus):                  0.554   Jarque-Bera (JB):                1.020
Skew:                          -0.384   Prob(JB):                        0.601
Kurtosis:                       2.770   Cond. No.                         134.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2. Métricas sobre el Modelo Completo y Modelo Reducido

Debido a que cuentan con diferentes número de parámetros, la comparación entre ellos se realiza en base a sus valores en $R^2_{adj}$, en la que se muestra que el modelo completo obtiene una mejor métrica en comparación con el reducido; por lo que también tiene una mejor métrica $R^2$ aunque tenga varias variables que no tienen un aporte significativo al modelo (o no se encuentran relacionadas linealmente con la calidad del vino). Esto último hace que el modelo completo sea mejor para predecir (tiene un mejor ajuste) aunque varias de sus variables (coeficientes de regresión) no sean significativas para el modelo o variable de respuesta, por lo que se podría reducir para mejorar la calidad de sus métricas.

In [18]:
ModeloReducido = smf.ols(
    f"{Target} ~ Aroma + Sabor",
    DataVinos
).fit()

ModeloReducido.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Calidad   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     33.75
Date:                Thu, 18 Sep 2025   Prob (F-statistic):           6.81e-09
Time:                        08:44:58   Log-Likelihood:                -60.188
No. Observations:                  38   AIC:                             126.4
Df Residuals:                      35   BIC:                             131.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.3462      1.009      4.307      0.000       2.298       6.395
Aroma          0.5180      0.276      1.877      0.069      -0.042       1.078
Sabor          1.1702      0.291      4.027      0.000       0.580       1.760
==============================================================================
Omnibus:                        0.321   Durbin-Watson:                   0.869
Prob(Omnibus):                  0.852   Jarque-Bera (JB):                0.499
Skew:                           0.076   Prob(JB):                        0.779
Kurtosis:                       2.460   Cond. No.                         35.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
print(f"R^2 del Modelo Completo :: {ModeloCompleto.rsquared}")
print(f"R^2_adj del Modelo Completo :: {ModeloCompleto.rsquared_adj}")

R^2 del Modelo Completo :: 0.720599218128954
R^2_adj del Modelo Completo :: 0.676942845961603


In [19]:
print(f"R^2 del Modelo Reducido :: {ModeloReducido.rsquared}")
print(f"R^2_adj del Modelo Reducido :: {ModeloReducido.rsquared_adj}")

R^2 del Modelo Reducido :: 0.6585515208879212
R^2_adj del Modelo Reducido :: 0.6390401792243738


## 3. Intervalo de Confianza para el Coeficiente de Regresión en Sabor

Aunque ambos modelos generan intervalos relativamente grandes, se tiene que el modelo reducido genera un intervalo más pequeño debido a que la varianza asociada al coeficiente de sabor es más pequeña (consecuencia del número de variables que se están usando) y su valor crítico ($t_{1-\alpha/2}$) también es más pequeño debido al incremento de los grados de libertad. Estos dos factores hacen que se genere un intervalo de confianza más reducido o más preciso para la estimación del coeficiente de sabor en el modelo reducido y con menos influencia de la propia varianza.

In [43]:
_IntervaloConfianzaCompleto = ModeloCompleto.conf_int(alpha=0.05).loc[['Sabor']]
_IntervaloConfianzaCompleto.rename(columns={0:'Límite Izquierdo',1:'Límite Derecho'},inplace=True)
_IntervaloConfianzaCompleto['Tamaño de Intervalo'] = _IntervaloConfianzaCompleto['Límite Derecho'] - _IntervaloConfianzaCompleto['Límite Izquierdo']

_IntervaloConfianzaCompleto

,Límite Izquierdo,Límite Derecho,Tamaño de Intervalo
Sabor,0.548117,1.788531,1.240414


In [44]:
_IntervaloConfianzaReducido = ModeloReducido.conf_int(alpha=0.05).loc[['Sabor']]
_IntervaloConfianzaReducido.rename(columns={0:'Límite Izquierdo',1:'Límite Derecho'},inplace=True)
_IntervaloConfianzaReducido['Tamaño de Intervalo'] = _IntervaloConfianzaReducido['Límite Derecho'] - _IntervaloConfianzaReducido['Límite Izquierdo']

_IntervaloConfianzaReducido

,Límite Izquierdo,Límite Derecho,Tamaño de Intervalo
Sabor,0.58033,1.760003,1.179674


## 4. Selección del Modelo por Pasos (Algoritmo Stepwise)

Usando el Criterio de Información de Akaike (CIA) para comparar entre dos modelos, se obtiene que el mejor modelo (según el algoritmo stepwise) se encuentra al usar las variables: Sabor, Fuerza y Aroma. Que a partir de sus p-valores asociados a sus respectivos t-valores, se tiene que todas las variables son significativas en el modelo, es decir, la calidad de un vino se encuentra al determinar su valor en sabor, fuerza y aroma, propiedades que concuerdan cuando uno piensa en cómo medir la calidad de un vino (principalmente el sabor y aroma), con ello se tiene que es conciso el resultado con lo se puede contrastar en el mundo real.

In [90]:
from copy import deepcopy

def StepwiseSelection(
        Dataset: pd.DataFrame,
        FeatureLabels: list[str],
        TargetLabel: str,
    ) -> list[str]: 
    """
    Algoritmo para la selección por pasos
    """
    
    CreateModelInstance = GenerateModels(Dataset,TargetLabel)
    BestLinearModel = smf.ols(
        f"{TargetLabel} ~ 1",
        data = Dataset
    ).fit()
    BestScore = EvaluateModel(BestLinearModel)
    BestFeatures = []

    while True:
        best_score = BestScore
        trial_features = deepcopy(BestFeatures)

        best_add_feature = ''
        for feature in [_feature for _feature in FeatureLabels if _feature not in trial_features]:
            LinearModel_Alt = CreateModelInstance(trial_features+[feature])
            score = EvaluateModel(LinearModel_Alt)
            
            if score < best_score:
                best_add_feature = feature
                best_score = score

        if best_add_feature:
            print(f"ADD :: {best_add_feature}")
            trial_features.append(best_add_feature)

        worst_remove_feature = ''
        for feature in trial_features:
            subset_features = [_feature for _feature in trial_features if _feature != feature]
            if subset_features:
                LinearModel_Alt = CreateModelInstance(subset_features)
                score = EvaluateModel(LinearModel_Alt)

                if score < best_score:
                    worst_remove_feature = feature
                    best_score = score

        if worst_remove_feature: 
            print(f"REMOVE :: {worst_remove_feature}")
            trial_features.remove(worst_remove_feature)

        LinearModel_Alt = CreateModelInstance(trial_features)
        best_score = EvaluateModel(LinearModel_Alt)
        if best_score < BestScore:
            BestFeatures = deepcopy(trial_features)
            BestScore = best_score
        else:
            break
    
    return BestFeatures

def GenerateModels(
        Dataset: pd.DataFrame,
        TargetLabel: str,
    ):
    """
    Función para facilitar la creación y 
    generación de los modelos en base a 
    un conjunto de datos.
    """

    def CreateModelInstance(
            FeaturesModel: list[str]
        ):
        """
        Función que crea los modelos en 
        base a los atributos que se 
        están considerando
        """
        if FeaturesModel:
            LinearModel = smf.ols(
                f"{TargetLabel} ~ " + ' + '.join(FeaturesModel),
                data = Dataset,
            ).fit()
        else:
            LinearModel = smf.ols(
                f"{TargetLabel} ~ 1",
                data = Dataset,
            ).fit()

        return LinearModel
    
    return CreateModelInstance

def EvaluateModel(
        LinearModel
    ) -> float:
    """
    Función para obtener la métrica de evaluación de un modelo. 
    Esta métrica se está minimizando por los diferentes métodos.
    """ 

    return np.e**(LinearModel.df_model/LinearModel.nobs) * LinearModel.mse_resid/LinearModel.nobs

In [91]:
FeatureLabels = ['Claridad' , 'Aroma' , 'Cuerpo' , 'Sabor' , 'Fuerza']

best_features_stepwise = StepwiseSelection(DataVinos,FeatureLabels,Target)
print(f"Las mejores variables son :: {best_features_stepwise}")

ADD :: Sabor
ADD :: Fuerza
ADD :: Aroma
Las mejores variables son :: ['Sabor', 'Fuerza', 'Aroma']


In [92]:
ModeloMejor = GenerateModels(DataVinos,Target)(best_features_stepwise)
ModeloMejor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Calidad   R-squared:                       0.704
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     26.92
Date:                Thu, 18 Sep 2025   Prob (F-statistic):           4.20e-09
Time:                        09:19:56   Log-Likelihood:                -57.489
No. Observations:                  38   AIC:                             123.0
Df Residuals:                      34   BIC:                             129.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.4672      1.333      4.852      0.000       3.759       9.176
Sabor          1.1997      0.275      4.364      0.000       0.641       1.758
Fuerza        -0.6023      0.264     -2.278      0.029      -1.140      -0.065
Aroma          0.5801      0.262      2.213      0.034       0.047       1.113
==============================================================================
Omnibus:                        0.955   Durbin-Watson:                   0.837
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.964
Skew:                          -0.338   Prob(JB):                        0.618
Kurtosis:                       2.611   Cond. No.                         58.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""